<a href="https://colab.research.google.com/github/Jahid1990/Deep-Learning-Applications-R-Python/blob/master/pack_reco_engin_allpacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import keras
import numpy as np
from keras import layers
from keras.layers import Input, LSTM, Dense,Conv1D,MaxPooling1D,Flatten,Embedding
from keras.layers import TimeDistributed
from keras.models import Model
from keras import metrics
from sklearn import preprocessing
import pickle

In [2]:
import math
from sklearn.model_selection import train_test_split
with open(r'/content/drive/My Drive/training_data_pack_reco_allpacks.pickle', 'rb') as f:
  data= pickle.load(f).sample(n=2500000)
  x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,:169],data.iloc[:,169], test_size=0.3, random_state=42)
  print(" Shape of X train:")
  print(x_train.shape)
  print(" Shape of y train:")
  print(y_train.shape)
  print(" Shape of X test:")
  print(x_test.shape)
  print(" Shape of y test:")
  print(y_test.shape)
  del data

 Shape of X train:
(1750000, 169)
 Shape of y train:
(1750000,)
 Shape of X test:
(750000, 169)
 Shape of y test:
(750000,)


In [3]:
#data.groupby(['RECHARGE_AMOUNT']).sample(n=2,replace=True)

In [10]:
# import math
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(pickle.load(f).iloc[:,:169],pickle.load(f).iloc[:,169], test_size=0.3, random_state=42)
# print(" Shape of X train:")
# print(x_train.shape)
# print(" Shape of y train:")
# print(y_train.shape)
# print(" Shape of X test:")
# print(x_test.shape)
# print(" Shape of y test:")
# print(y_test.shape)
# del data

In [9]:
y_train=pd.get_dummies(y_train)
y_test=pd.get_dummies(y_test)

In [10]:
scaler = preprocessing.MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [86]:
from keras.layers import Concatenate
inputs = Input(shape=(x_train.shape[1],))
pack=Dense(1024,activation='relu')(inputs)
pack=Dense(1024,activation='relu')(pack)
pack=Dense(1024,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='tanh')(pack)
pack=Dense(y_train.shape[1],activation='softmax',name='pack')(pack)
model=Model(inputs=inputs,outputs=[pack])
#model.summary()

In [87]:
from keras import optimizers
adm=optimizers.Adam( lr=0.0001,beta_1=0.9, beta_2=0.999, amsgrad=False)

In [88]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
checkpoint = ModelCheckpoint('/content/drive/My Drive/pack_reco_engine_allpacks.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

In [5]:
import keras.backend as K
import tensorflow as tf

def loss(y_true, y_pred):
  y_true=K.cast(y_true,'float32')
  y_pred=K.cast(y_pred,'float32')
  # scale predictions so that the class probas of each sample sum to 1
  y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
  # clip to prevent NaN's and Inf's
  y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
  # calc
  loss = y_true * K.log(y_pred) * K.sum(y_true,axis=1, keepdims=True)
  loss = -K.sum(loss, -1)
  return loss

In [4]:
def loss(y_true, y_pred):
    y_true=K.cast(y_true,'float32')
    y_pred=K.cast(y_pred,'float32')
    gamma = 2.0 
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


In [91]:
from datetime import datetime
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
model.compile(loss=loss,optimizer=adm, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100,validation_data=(x_test,y_test),verbose=1,batch_size=512,callbacks=[checkpoint,tensorboard_callback])

In [ ]:
!kill 2160
%load_ext tensorboard
%tensorboard --logdir logs/scalars

In [ ]:
custom_objects={'my_custom_func':                   
my_custom_func}

In [7]:
import numpy as np
import tensorflow as tf
np.set_printoptions(suppress=True)
model=tf.keras.models.load_model("/content/drive/My Drive/pack_reco_engine_allpacks.h5",custom_objects={'loss':loss})
p=model.predict(x_test)

In [80]:
print(classification_report(pred_class,actual))

              precision    recall  f1-score   support

           0       0.86      0.74      0.79    416434
           1       0.02      0.26      0.04       308
           2       0.41      0.62      0.49     29750
           3       0.20      0.31      0.25      4534
           4       0.42      0.50      0.46     22504
           5       0.01      0.08      0.02       663
           6       0.09      0.14      0.11      7751
           7       0.31      0.54      0.39      4388
           8       0.42      0.45      0.44     42994
           9       0.66      0.65      0.65     73073
          10       0.46      0.58      0.51     21015
          11       0.28      0.46      0.35      9001
          12       0.02      0.11      0.04       251
          13       0.10      0.21      0.14       870
          14       0.55      0.71      0.62     22401
          15       0.41      0.42      0.41     15336
          16       0.59      0.70      0.64      5602
          17       0.39    

In [11]:
pred_class=np.argmax(p, axis=1)
actual=np.argmax(y_test.to_numpy(),axis=1)
from sklearn.metrics import classification_report
print(classification_report(pred_class,actual))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.48      0.65    749495
           1       0.00      0.00      0.00         3
           2       0.00      0.22      0.00       200
           3       0.00      0.07      0.00        28
           4       0.00      0.11      0.00        18
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00        10
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          10       0.00      0.33      0.00         3
          11       0.00      0.30      0.00        53
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         0
          16       0.00      0.48      0.01        61
          17       0.00    

In [57]:
pd.Series(y_test).value_counts()

ValueError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=10,verbose=1)
clf.fit(x_train,y_train)



In [64]:
len(pd.Series(pred_class).value_counts())

31

In [12]:
top3_packs=np.argsort(-p)[:,:3]
eval=[]
for i in actual:
  if pd.Series(i).isin(top3_packs[i,:]).bool():
    eval.append(1) 
  else:
    eval.append(0)


In [13]:
pd.Series(eval).value_counts()/len(eval)

1    0.632661
0    0.367339
dtype: float64

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(eval,actual))